In [288]:
import numpy as np

## Gameplay

In [323]:
class Gameplay:
    def __init__(self, n_rows, n_cols):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.rules = []
        self.tiles = np.zeros((n_rows, n_cols), dtype=object)
        for i in range(n_rows):
            for j in range(n_cols):
                self.tiles[i,j] = Tile()

        self.prefixes = ['baba', 'rock', 'water', 'skull', 'wall', 'flag']
        self.suffixes = ['win', 'defeat', 'sink', 'you', 'push', 'stop']        

    def __repr__(self):
        ret_val = ''
        for i in range(self.n_rows):
            for j in range(self.n_cols):
                tile_string = ''
                if len(self.tiles[i,j].objects) == 0:
                    tile_string += '/'
                else:
                    for obj in self.tiles[i,j].objects:
                        if isinstance(obj, Word):
                            tile_string += obj.value.upper()
                        else:
                            tile_string += obj.property if obj.property !='' else 'null'
                        tile_string += ','
                ret_val += '{:10}'.format(tile_string)
            ret_val += '\n'           
        return ret_val

    
    def get_rules(self):
        self.rules = []
        for i in range(self.n_rows):
            for j in range(self.n_cols):
                if self.tiles[i,j].find_word() == 'is':
                    # horizontal
                    if (j-1 >= 0) and (j+1 <= self.n_cols-1) and (self.tiles[i,j-1].find_word() in self.prefixes) and (self.tiles[i,j+1].find_word() in self.suffixes):
                        self.rules += [Rule(self.tiles[i,j-1].find_word(), self.tiles[i,j+1].find_word())]

                    # verical
                    if (i-1 >= 0) and (i+1 <= self.n_rows-1) and (self.tiles[i-1,j].find_word() in self.prefixes) and (self.tiles[i+1,j].find_word() in self.suffixes):
                        self.rules += [Rule(self.tiles[i-1,j].find_word(), self.tiles[i+1,j].find_word())]

    # find a block upward (continuous)
    # return block size if the block can be moved up
    # return -1 if the block is blocked
    def find_up_block(self, r, c):
        if not self.tiles[r,c].have_property('you'):
            return 0
        
        current_row = r - 1 #dem row tu 0 nen phai -1
        count = 1

        if current_row < 0: #khong day block len duoc nua
            return -1

        while current_row >= 0 and self.tiles[current_row,c].have_property('push'): #trong truong hop day duoc block thi dem row
            count += 1
            current_row -= 1
        
        if current_row < 0 or self.tiles[current_row,c].have_property('stop') or self.tiles[current_row,c].have_property('you'): #check o tiep theo cua block co phai limit cua map hoac chua stop hay khong
            return -1
        else:
            return count

    def find_down_block(self, r, c):
        if not self.tiles[r,c].have_property('you'):
            return 0

        current_row = r + 1
        count = 1

        if current_row >= self.n_rows:
            return -1
        while current_row < self.n_rows and self.tiles[current_row,c].have_property('push'):
            count += 1
            current_row += 1
        if current_row >= self.n_rows or self.tiles[current_row,c].have_property('stop') or self.tiles[current_row,c].have_property('you'):
            return -1
        else:
            return count  

    def find_right_block(self, r, c):
        if not self.tiles[r,c].have_property('you'):
            return 0

        current_col = c + 1
        count = 1

        if current_col >= self.n_rows:
            return -1
        while current_col < self.n_rows and self.tiles[r,current_col].have_property('push'):
            count += 1
            current_col += 1
        if current_col >= self.n_rows or self.tiles[r,current_col].have_property('stop') or self.tiles[r,current_col].have_property('you'):
            return -1
        else:
            return count  

    def find_left_block(self, r, c):
        if not self.tiles[r,c].have_property('you'):
            return 0
        
        current_col = c - 1
        count = 1

        if current_col < 0:
            return -1

        while current_col >= 0 and self.tiles[r,current_col].have_property('push'):
            count += 1
            current_col -= 1
        
        if current_col < 0 or self.tiles[r,current_col].have_property('stop') or self.tiles[r,current_col].have_property('you'):
            return -1
        else:
            return count
        
    def move_up(self):
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                size = self.find_up_block(r,c)
                if size > 0:
                    for current_row in range(r - size + 1, r + 1):
                        temp = self.tiles[current_row,c].pop_push_or_you()
                        self.tiles[current_row-1,c].objects = np.append(self.tiles[current_row-1,c].objects,[temp])

    def move_down(self):
        for r in range(self.n_rows-1, -1, -1):
            for c in range(self.n_cols):
                size = self.find_down_block(r,c)
                if size > 0:
                    for current_row in range(r + size - 1, r-1, -1):
                        temp = self.tiles[current_row,c].pop_push_or_you()
                        self.tiles[current_row+1,c].objects = np.append(self.tiles[current_row+1,c].objects,[temp])

    # move_left giống move_up
    def move_left(self):
        for c in range(self.n_cols):
            for r in range(self.n_rows):
                size = self.find_left_block(r,c)
                if size > 0:
                    for current_col in range(c - size + 1, c + 1):
                        temp = self.tiles[r,current_col].pop_push_or_you() # lưu ô hiện tại vào temp
                        self.tiles[r,current_col-1].objects = np.append(self.tiles[r,current_col-1].objects,[temp]) # nhét temp vào cột bên trái

    # move_right giống move_down
    def move_right(self):
        for c in range(self.n_cols-1, -1, -1):
            for r in range(self.n_rows):
                size = self.find_right_block(r,c)
                if size > 0:
                    for current_col in range(c + size - 1, c-1, -1):
                        temp = self.tiles[r,current_col].pop_push_or_you()
                        self.tiles[r,current_col+1].objects = np.append(self.tiles[r,current_col+1].objects,[temp])

    def check_win(self):
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                if self.tiles[r,c].have_property('you') and self.tiles[r,c].have_property('win'):
                    return True
        return False
    
    def check_lose(self):
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                if self.tiles[r,c].have_property('you'):
                    return False
        return True

    def reset_game(self):
        pass

    def load_map(self, file_name):
        self.file_name = file_name

## Rules

In [290]:
class Rule:
    def __init__(self, first, second):
        # first là giá trị trc chữ 'is', second là giá trị sau chữ 'is'
        self.first = first
        self.second = second

    def __repr__(self):
        return self.first + ' is ' + self.second

## Object

In [291]:
class Object:
    def __init__(self, property=''):
        self.property = property

    def interact(self, another_object: object):
        pass

In [292]:
class Baba(Object):
    def interact(self, another_object: object):
        pass

In [293]:
class Rock(Object):
    def interact(self, another_object: object):
        pass    

In [294]:
class Water(Object):
    def interact(self, another_object: object):
        pass

In [295]:
class Skull(Object):
    def interact(self, another_object: object):
        pass    

In [296]:
class Wall(Object):
    def interact(self, another_object: object):
        pass

In [297]:
class Word(Object):
    def __init__(self, value):
        super().__init__("push")
        self.value = value
    
    def interact(self, another_object: object):
        pass
        

In [298]:
class Flag(Object):
    def interact(self, another_object: object):
        pass

## Tiles

In [299]:
class Tile:
    def __init__(self):
        self.objects = np.array([], dtype=object)

    def add_object(self, obj: Object):
        self.objects = np.append(self.objects, [obj])

    def find_word(self):
        # return word value if exists, return '' otherwise
        for i in self.objects:
            if isinstance(i, Word):
                return i.value
        return ''

    def have_property(self, property):
        for obj in self.objects:
            if obj.property == property:
                return True
        return False

    def pop_push_or_you(self):
        position = -1
        for i in range(len(self.objects)):
            if self.objects[i].property == 'push' or self.objects[i].property == 'you':
                position = i
        temp_object = self.objects[position]
        self.objects = np.delete(self.objects, position)


        return temp_object

## Unit Testing

In [348]:
gp = Gameplay(10,10)
gp.tiles[0,0].add_object(Word('baba'))
gp.tiles[0,1].add_object(Word('baba'))
gp.tiles[1,0].add_object(Word('is'))
gp.tiles[2,2].add_object(Word('rock'))
#gp.tiles[1,2].add_object(Baba('you'))
#gp.tiles[5,5].add_object(Baba('you'))
gp.tiles[9,9].add_object(Flag('win'))
gp.get_rules()

In [349]:
gp.move_up()
print(gp)

BABA,     BABA,     /         /         /         /         /         /         /         /         
IS,       /         /         /         /         /         /         /         /         /         
/         /         ROCK,     /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         w

In [346]:
gp.move_down()
print(gp)


BABA,     BABA,     /         /         /         /         /         /         /         /         
IS,       /         /         /         /         /         /         /         /         /         
/         /         ROCK,     /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         you,      /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         w

In [322]:
gp.move_left()
print(gp)

BABA,     BABA,     /         /         /         /         /         /         /         /         
IS,       you,      /         /         /         /         /         /         /         /         
/         /         ROCK,     /         /         /         /         /         /         /         
/         you,      /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         w

In [341]:
gp.move_right()
print(gp)

BABA,     BABA,     /         /         /         /         /         /         /         /         
IS,       /         /         /         /         /         you,      /         /         /         
/         /         ROCK,     /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         you,      
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         w

In [350]:
if gp.check_win():
    print('You win')
if gp.check_lose():
    print('You are fucking dead bro, you suck')

You are fucking dead bro, you suck
